<a href="https://colab.research.google.com/github/nikpournastaran/Unina-Thesis/blob/main/ASR_LLM_TTS_Pipeline_(Italiano_with_EN_Comments).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**0. INSTALLATIONS**

In [ ]:
# Run this cell first to install all necessary libraries.
print("Installing necessary libraries...")
!pip install -U openai-whisper jiwer pydub
!pip install transformers sentencepiece
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!apt install ffmpeg
print("✅ Installation complete.")

Installing necessary libraries...
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.1.4
    Uninstalling huggingface_hub-1.1.4:
      Successfully uninstalled huggingface_hub-1.1.4
Looking in indexes: https://download.pytorch.org/whl/cu118
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
✅ Installation complete.


**1. FILE UPLOAD**

In [ ]:
from google.colab import files

print("Upload your audio file (e.g., M4A, MP3, WAV)...")
uploaded = files.upload()

# Save the name of the uploaded file
# We assume only one file is uploaded
input_file_name = list(uploaded.keys())[0]
print(f"✅ File uploaded: {input_file_name}")

Upload your audio file (e.g., M4A, MP3, WAV)...


Saving solo 50 Centesimi2.m4a to solo 50 Centesimi2.m4a
✅ File uploaded: solo 50 Centesimi2.m4a


**2. AUDIO CONVERSION TO WAV**

In [ ]:
# Whisper works best with the WAV format.
# We convert the uploaded file to 'test_audio.wav'.
from pydub import AudioSegment
import os

output_file_wav = "test_audio.wav"

try:
    print(f"Converting '{input_file_name}' to WAV...")
    # Determine the format from the file extension
    file_extension = os.path.splitext(input_file_name)[1].lower().replace('.', '')

    if not file_extension:
        # If there is no extension, try to load it directly
        audio = AudioSegment.from_file(input_file_name)
    else:
        # Use the specified format
        audio = AudioSegment.from_file(input_file_name, format=file_extension)

    # Export to WAV format
    audio.export(output_file_wav, format="wav")
    print(f"✅ File successfully converted to: {output_file_wav}")

except Exception as e:
    print(f"❌ Error during conversion: {e}")
    print("Make sure the file is a valid audio format and that FFmpeg is installed.")

Converting 'solo 50 Centesimi2.m4a' to WAV...
✅ File successfully converted to: test_audio.wav


**3. STAGE 1: ASR (Automatic Speech Recognition)**

In [ ]:
import whisper
print("\n--- Start STAGE 1: ASR (Whisper) ---")

# Load the model (e.g., 'small', 'medium', 'large')
# 'small' is a good balance between speed and accuracy
model = whisper.load_model("small")

# Transcribe the audio, specifying the Italian language
result = model.transcribe(output_file_wav, language="it")
asr_output = result["text"].strip()

print(f"ASR Output (Whisper): {asr_output}")


--- Start STAGE 1: ASR (Whisper) ---


/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


ASR Output (Whisper): Ieri io ho andato al negozio per comprare due pane.


**4. ASR ERROR ANALYSIS (WER)**

In [ ]:
from jiwer import wer
print("\n--- ASR Error Analysis (WER) ---")

# !!! IMPORTANT: You must modify this 'ground_truth' variable !!!
# Enter the correct (manual) transcription of your audio file here.
# Italian Example:
ground_truth = "Sono andato al mercato ieri."
# Example ASR output with error: "Sono andato mercato ieri"

hypothesis = asr_output

# Calculate the Word Error Rate
error_rate = wer(ground_truth, hypothesis)

print(f"Reference Text (GT): {ground_truth}")
print(f"Hypothesis (ASR): {hypothesis}")
print(f"ASR Word Error Rate (WER): {error_rate * 100:.2f}%")
print("(Lower is better)")



--- ASR Error Analysis (WER) ---
Reference Text (GT): Sono andato al mercato ieri.
Hypothesis (ASR): Ieri io ho andato al negozio per comprare due pane.
ASR Word Error Rate (WER): 160.00%
(Lower is better)


**4.5 Authenticate with Hugging Face**

In [ ]:
# --- 4.5 (NEW CELL A) - Upgrade Hugging Face Library ---
# Run this cell ONE TIME to install the latest version of huggingface_hub.
print("\n--- Upgrading Hugging Face Hub ---")
!pip install huggingface_hub -U
print("✅ Upgrade complete.")
print("‼️ IMPORTANT: You MUST restart the runtime now before continuing.")
print("Go to the menu: Runtime > Restart runtime...")


--- Upgrading Hugging Face Hub ---
✅ Upgrade complete.
‼️ IMPORTANT: You MUST restart the runtime now before continuing.
Go to the menu: Runtime > Restart runtime...


**4.6 (NEW CELL B) - Authenticate with Hugging Face**

In [ ]:
# AFTER restarting the runtime, run THIS cell.
# It will ask you for your token directly in the output below.

print("\n--- Authenticating with Hugging Face (CLI Method) ---")
print("A 'Token' prompt will appear below.")
print("1. Paste your token (hf_...) into the prompt.")
print("2. Press Enter.")
print("3. It will ask 'Add token as git credential? (Y/n)'. Type 'n' and press Enter.")

!huggingface-cli login


--- Authenticating with Hugging Face (CLI Method) ---
A 'Token' prompt will appear below.
1. Paste your token (hf_...) into the prompt.
2. Press Enter.
3. It will ask 'Add token as git credential? (Y/n)'. Type 'n' and press Enter.
/bin/bash: line 1: huggingface-cli: command not found


**5. STAGE 2: LLM (Text Correction)**

In [ ]:
# --- 5. STAGE 2: LLM (Text Correction) ---
from transformers import pipeline
print("\n--- Start STAGE 2: LLM (Text Correction) ---")

# We use a model specialized for Italian grammar correction
# (it5-base is lighter than larger models)
print("Loading LLM model for correction (this may take time)...")
try:
    # --- FIX: Corrected the model identifier from 'it5/' to 'gsarti/' ---
    corrector = pipeline("text2text-generation", model="gsarti/it5-base-grammar-correction")
    print("✅ LLM model loaded.")

    input_to_llm = hypothesis # We use the output from ASR

    # The model expects a "correggi:" prefix
    llm_result = corrector(f"correggi: {input_to_llm}")

    llm_output = llm_result[0]['generated_text'].strip()

    print(f"Input to LLM (from ASR): {input_to_llm}")
    print(f"LLM Output (Corrected): {llm_output}")

except Exception as e:
    print(f"❌ Error during LLM loading or execution: {e}")
    print("Fallback: using original ASR output for TTS.")
    llm_output = hypothesis # Fallback

ImportError: huggingface-hub>=0.34.0,<1.0 is required for a normal functioning of this module, but found huggingface-hub==1.1.4.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

**6. POST-LLM ERROR ANALYSIS (WER)**

In [ ]:
print("\n--- Post-LLM Error Analysis (WER) ---")

# We compare the corrected output from the LLM with the original GT
reference_for_llm = ground_truth
hypothesis_for_llm = llm_output

# Calculate WER after correction
llm_error_rate = wer(reference_for_llm, hypothesis_for_llm)

print(f"Reference Text (GT): {reference_for_llm}")
print(f"Corrected Text (LLM): {hypothesis_for_llm}")
print(f"WER after LLM correction: {llm_error_rate * 100:.2f}%")
print("(This value is expected to be lower than the initial ASR WER)")


--- Post-LLM Error Analysis (WER) ---
Reference Text (GT): Sono andato al mercato ieri.
Corrected Text (LLM): Ieri io ho andato al negozio per comprare due pane.
WER after LLM correction: 160.00%
(This value is expected to be lower than the initial ASR WER)


**7. STAGE 3: TTS (Text-to-Speech)**

In [ ]:
# Install gTTS (Google Text-to-Speech)
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio, display
print("\n--- Start STAGE 3: TTS (Speech Synthesis) ---")

input_to_tts = llm_output
tts_mp3_file = "tts_output_temp.mp3"
tts_wav_file = "tts_output_finale.wav"

try:
    # 1. Generate MP3 file with gTTS in Italian
    print("Generating TTS audio...")
    tts = gTTS(text=input_to_tts, lang='it', slow=False)
    tts.save(tts_mp3_file)
    print(f"✅ Temporary MP3 file generated: {tts_mp3_file}")

    # 2. Convert MP3 to WAV (necessary for some environments)
    audio = AudioSegment.from_mp3(tts_mp3_file)
    audio.export(tts_wav_file, format="wav")
    print(f"✅ Final WAV file generated: {tts_wav_file}")

    # 3. Remove the temporary MP3 file
    os.remove(tts_mp3_file)

    # 4. Play the final audio
    print("\n🎧 Playing the final corrected audio:")
    display(Audio(tts_wav_file))

except Exception as e:
    print(f"❌ Error during TTS generation or conversion: {e}")

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/index_command.py", line 100, in _build_session
    session = PipSession(
              ^^^^^^^^^